# 당신의 사상의학 체질은?

## 1. 데이터 전처리
## 2. Features에 점수 부여하기
## 3. 모델링
## 4. 결론

## 1. 데이터 전처리

In [315]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns ; sns.set()

In [340]:
import warnings; warnings.simplefilter('ignore')

In [341]:
data = pd.read_excel('LEE_JAEMA.xlsx')

In [342]:
df = pd.DataFrame(data)

In [343]:
df.rename(columns={'타임스탬프':'time', "0-1. 귀하의 성별은 어떻게되나요?":'gender', '0-2. 귀하의 나이는 어떻게되나요?':'age',
                  '1-1.귀하의 성격상의 장점은 무엇인가요? (자라면서성격이 바뀌었다면 바뀌기 전에 본연의 성향 중 가장 많이 해당하는 것으로 한가지만 선택하세요)':"good",
                  '1-2. 귀하의 성격상의 단점은 무엇인가요? (자라면서성격이 바뀌었다면 바뀌기 전에 본연의 성향 중 가장 많이 해당하는 것으로 한가지만 선택하세요)':'bad',
                  '2-1. 귀하의 평소 경향성은 무엇인가요? (평소 자신의 성향에 가장 많이 해당하는 것을 한 가지만 선택하세요)':'tendency','3-1. 귀하가 아래의 음식 중, 평소 섭취 후 ‘두 번 중 한 번 이상 소화불량, 설사, 복통, 변비, 불면, 가슴 두근거림, 알레르기’ 등의 이상반응이 있는 개수는?':'food1',
                  '3-2.귀하가 아래의 음식 중, 평소 섭취 후 ‘두 번 중 한 번 이상 소화불량, 설사, 복통, 변비, 불면, 가슴 두근거림, 알레르기’ 등의 이상반응이 있는 개수는?':'food2',
                  '4-1. 귀하는 식사시 한번에 섭취하는 음식량이 어떻게 되나요?':'pig',
                  '5-1. 귀하는 평소 땀을 얼마나 흘리나요? (가장가깝다고 생각되는 선택지를 선택해주세요)':'sweat',
                  '6-1. 귀하의 평소 💩상태는 어떤가요? (가장가깝다고 생각되는 선택지를 선택해주세요)':'big',
                  '7-1. 귀하의 소변상태는 어떤가요? (가장가깝다고 생각되는 선택지를 선택해주세요)':'small',
                  '8-1. 귀하의 전반적인 몸 상태는 어떤가요? (가장가깝다고 생각되는 선택지를 선택해주세요)':'body',
                  '9-1. 귀하의 신체 골격은 아래의 사진 중 어디에 제일 가까운 것 같나요? (가장가깝다고 생각되는 선택지를 선택해주세요)':'bone',
                  },inplace=True)

In [344]:
df.drop(['time', 'gender','age','감사합니다 😆 이메일을 남겨주세요~'], axis='columns', inplace=True)

In [345]:
df.loc[ df['good'].str.contains('이성적'), 'good' ] = '2'

In [346]:
df.loc[ df['good'].str.contains('원만함'), 'good' ] = '1'
df.loc[ df['good'].str.contains('이성적'), 'good' ] = '2'
df.loc[ df['good'].str.contains('아이디어'), 'good' ] = '3'
df.loc[ df['good'].str.contains('진취적'), 'good' ] = '4'

df.loc[ df['bad'].str.contains('욕심'), 'bad' ] = '1'
df.loc[ df['bad'].str.contains('내성적'), 'bad' ] = '2'
df.loc[ df['bad'].str.contains('가벼움'), 'bad' ] = '3'
df.loc[ df['bad'].str.contains('독선'), 'bad' ] = '4'

df.loc[ df['tendency'].str.contains('특정한'), 'tendency' ] = '3'
df.loc[ df['tendency'].str.contains('사회의'), 'tendency' ] = '2'
df.loc[ df['tendency'].str.contains('재물'), 'tendency' ] = '1'
df.loc[ df['tendency'].str.contains('예의범절이나'), 'tendency' ] = '4'

df.loc[ df['food1'].str.contains('4개'), 'food1' ] = '1'
df.loc[ df['food1'].str.contains('1~3개'), 'food1' ] = '2'
df.loc[ df['food1'].str.contains('해당사항 없음'), 'food1' ] = '3'

df['food2']=df['food2'].fillna('3')
df.loc[ df['food2'].str.contains('1 ~ 3개'), 'food2' ] = '2'
df.loc[ df['food2'].str.contains('해당사항 없음'), 'food2' ] = '3'

df.loc[ df['pig'].str.contains('한끼에'), 'pig' ] = '1'
df.loc[ df['pig'].str.contains('적게'), 'pig' ] = '2'

df.loc[ df['sweat'].str.contains('땀을'), 'sweat' ] = '1'
df.loc[ df['sweat'].str.contains('운동을'), 'sweat' ] = '2'
df.loc[ df['sweat'].str.contains('더위를'), 'sweat' ] = '3'

df['big']=df['big'].fillna('2')
df.loc[ df['big'].str.contains('설사가'), 'big' ] = '1'
df.loc[ df['big'].str.contains('굵다'), 'big' ] = '2'
df.loc[ df['big'].str.contains('변비가'), 'big' ] = '3'


df.loc[ df['small'].str.contains('자신이'), 'small' ] = '1'
df.loc[ df['small'].str.contains('않다'), 'small' ] = '2'

df.loc[ df['body'].str.contains('차고'), 'body' ] = '1'
df.loc[ df['body'].str.contains('더위를'), 'body' ] = '2'



## 2. Features에 점수 부여하기

In [347]:
df['good'].replace('1', 13,inplace=True)
df['good'].replace('2', 18,inplace=True)
df['good'].replace('3', 8,inplace=True)
df['good'].replace('4', 3,inplace=True)

df['bad'].replace('1', 12,inplace=True)
df['bad'].replace('2', 17,inplace=True)
df['bad'].replace('3', 7,inplace=True)
df['bad'].replace('4', 2,inplace=True)

df['character']=df['good']+df['bad']

In [348]:
df['tendency'].replace('3', 5,inplace=True)
df['tendency'].replace('2', 15,inplace=True)
df['tendency'].replace('1', 10,inplace=True)
df['tendency'].replace('4', 0,inplace=True)

In [349]:
df['food_total'] = 0
df.loc[df['food1']=='1','food1']=50
df.loc[df['food1']=='2', 'food1']=40
df.loc[df['food2']=='2', 'food2']=20
df.loc[df['food2']=='1','food2']=10
df.loc[df['food1']=='3','food1']=25
df.loc[df['food2']=='3','food2']=5
df.loc[(df['food1']=='3')&(df['food2']=='3'),'food_total']=30
df.loc[(df['food1']=='1')&(df['food2']=='1'),'food_total']=50


df.loc[(df['food_total']==0)&((df['character']+df['tendency'])< 35)&(df['food1']>df['food2']), 'food_total']= df['food2']
df.loc[(df['food_total']==0)&((df['character']+df['tendency'])< 35)&(df['food1']<=df['food2']), 'food_total']= df['food1']
df.loc[(df['food_total']==0)&((df['character']+df['tendency'])>= 35)&(df['food1']>df['food2']), 'food_total']= df['food1']
df.loc[(df['food_total']==0)&((df['character']+df['tendency'])>= 35)&(df['food1']<=df['food2']), 'food_total']= df['food2']

In [350]:
df.head()

,good,bad,tendency,food1,food2,pig,sweat,big,small,body,bone,character,food_total
0,18,12,5,25,5,1,1,1,1,2,2,30,25
1,18,17,5,40,5,2,2,3,1,1,3,35,40
2,8,7,15,25,5,1,1,2,1,2,3,15,5
3,18,12,5,25,5,1,3,2,1,2,2,30,25
4,13,12,10,25,5,1,3,1,1,2,2,25,25


In [351]:
df['pig'].replace('1', 10,inplace=True)
df['pig'].replace('2', 20,inplace=True)

df['sweat'].replace('2', 30,inplace=True)
df['sweat'].replace('1', 20,inplace=True)
df['sweat'].replace('3', 10,inplace=True)

df['big'].replace('1', 50,inplace=True)
df['big'].replace('2', 25,inplace=True)
df['big'].replace('3', 0,inplace=True)

In [352]:
df1 = df[{'food_total','character','tendency','pig','sweat','big'}]

In [353]:
df1['sum']=df1['food_total']+df1['character']+df1['tendency']+df1['pig']+df1['sweat']+df1['big']

In [354]:
df1.head()

,food_total,character,big,sweat,tendency,pig,sum
0,25,30,50,20,5,10,140
1,40,35,0,30,5,20,130
2,5,15,25,20,15,10,90
3,25,30,25,10,5,10,105
4,25,25,50,10,10,10,130


In [355]:
df1.loc[df1['sum']<47.5,'result']='태양인'
df1.loc[(df1['sum']>=47.5)&(df1['sum']<97.5),'result']='소양인'
df1.loc[(df1['sum']>=97.5)&(df1['sum']<142.5),'result']='태음인'
df1.loc[df1['sum']>=142.5,'result']='소음인'

In [356]:
df1.head()

,food_total,character,big,sweat,tendency,pig,sum,result
0,25,30,50,20,5,10,140,태음인
1,40,35,0,30,5,20,130,태음인
2,5,15,25,20,15,10,90,소양인
3,25,30,25,10,5,10,105,태음인
4,25,25,50,10,10,10,130,태음인


## 3. 모델링

In [357]:
traing_data = df1[['food_total','character','tendency','big','pig','sweat']]
target_data = df1[['result']]

In [358]:
traing_data.head()

,food_total,character,tendency,big,pig,sweat
0,25,30,5,50,10,20
1,40,35,5,0,20,30
2,5,15,15,25,10,20
3,25,30,5,25,10,10
4,25,25,10,50,10,10


In [359]:
target_data.head()

,result
0,태음인
1,태음인
2,소양인
3,태음인
4,태음인


In [360]:
from sklearn.model_selection import train_test_split

In [361]:
X_train,X_test,y_train, y_test = train_test_split(traing_data,target_data)

### 3-1. 랜덤포레스트

In [362]:
from sklearn.ensemble import RandomForestClassifier

In [363]:
rf = RandomForestClassifier(criterion='entropy', n_estimators=150, random_state=0, max_depth=2)
rf_model = rf.fit(X_train, y_train)
y_pred_train = rf_model.predict(X_train)
y_pred_test = rf_model.predict(X_test)

In [364]:
print("훈련 세트 정확도: {:.2f}".format(rf_model.score(X_train, y_train)))
print("테스트 세트 정확도: {:.2f}".format(rf_model.score(X_test, y_test)))

훈련 세트 정확도: 0.79
테스트 세트 정확도: 0.68


### 3-2. 서포트벡터머신

In [365]:
from sklearn.svm import SVC

In [366]:
svc = SVC(kernel='rbf', class_weight='balanced')

In [367]:
svm = svc.fit(X_train, y_train)

In [368]:
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10, 50],
              'gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(svm, param_grid)

%time grid.fit(X_train, y_train)
print(grid.best_params_)

Wall time: 471 ms
{'C': 50, 'gamma': 0.0001}


최적 C(규제 매개변수) = 50, 최적 gamma(커널의 폭) = 0.0001

In [369]:
model = grid.best_estimator_
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

In [370]:
print("훈련 세트 정확도: {:.2f}".format(model.score(X_train, y_train)))
print("테스트 세트 정확도: {:.2f}".format(model.score(X_test, y_test)))

훈련 세트 정확도: 0.92
테스트 세트 정확도: 0.82


### 3-3. 로지스틱스

In [371]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
log_model = log_reg.fit(X_train, y_train)
y_pred_train = log_model.predict(X_train)
y_pred_test = log_model.predict(X_test)

In [372]:
print("훈련 세트 정확도: {:.2f}".format(log_model.score(X_train, y_train)))
print("테스트 세트 정확도: {:.2f}".format(log_model.score(X_test, y_test)))

훈련 세트 정확도: 0.79
테스트 세트 정확도: 0.73


## 4. 결론

### 서포트벡터 머신이 테스트 세트 정확도 0.82으로 가장 높음.

In [373]:
y_pred = model.predict(traing_data)

In [374]:
y_pred

array(['소음인', '태음인', '소양인', '태음인', '태음인', '태음인', '태음인', '태음인', '태음인',
       '태음인', '소양인', '태음인', '소음인', '소양인', '태음인', '태양인', '소음인', '소양인',
       '태음인', '소양인', '소양인', '태음인', '소양인', '소음인', '소음인', '소양인', '소양인',
       '소양인', '소양인', '태음인', '소음인', '소양인', '태양인', '태음인', '소음인', '태음인',
       '소음인', '태음인', '태음인', '태음인', '태음인', '태양인', '태음인', '소양인', '소음인',
       '태음인', '소양인', '소양인', '소양인', '태음인', '태음인', '소음인', '태음인', '소음인',
       '소음인', '태음인', '소양인', '태음인', '태음인', '태음인', '태음인', '태음인', '태음인',
       '태음인', '태음인', '소양인', '태음인', '소양인', '소양인', '소양인', '태음인', '태음인',
       '소음인', '태음인', '소양인', '태음인', '소음인', '소양인', '태음인', '소양인', '소양인',
       '태음인', '태음인', '소양인', '소음인', '태음인', '소음인', '태음인'], dtype=object)